In [41]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import glob

In [42]:
group10 = pd.read_csv('../Data/Target Data/group10_header.csv',
                      sep='\t', low_memory=False)

In [43]:
total_tcin = group10['tcin'].unique()
file_list = glob.glob("../Data/Target Data/scraped/*")
tcin_completed = [file.split('/')[-1].split('_')[0] for file in file_list]
tcin_completed.remove('failed')
tcin_completed_set = set(pd.Series(tcin_completed).astype('int'))
total_tcin_set = set(total_tcin)
tcin_left = sorted(total_tcin_set.difference(tcin_completed_set))

In [44]:
len(total_tcin_set)

23248

In [45]:
len(tcin_completed_set)

4062

In [46]:
len(tcin_left)

19186

In [ ]:
driver = webdriver.Chrome('/opt/homebrew/bin/chromedriver') 
driver.get('https://www.target.com/p/thomas-39-plain-bagel-thins-13oz-8ct/-/A-12921319#lnk=sametab')
failed_tcin = []

for tcin in tcin_left:
    try:
        driver.get(f'''https://www.target.com/p/-/A-{tcin}''')
        button = driver.find_element(By.XPATH, '//button[text()="Show more"]')
        driver.execute_script("arguments[0].click();", button)
        with open(f"../Data/Target Data/scraped/{tcin}_details.html", "w") as f:
            f.write(driver.page_source)
        link = driver.find_element_by_link_text('Label info')
        driver.execute_script("arguments[0].click();", link)
        with open(f"../Data/Target Data/scraped/{tcin}_label_info.html", "w") as f:
            f.write(driver.page_source)
    except:
        failed_tcin.append(tcin)
        
with open("../Data/Target Data/scraped/failed_tcin.txt", 'w') as f:
    f.write(str(failed_tcin))